In [64]:
import torch
from pathlib import Path
import pandas as pd
import numpy as np
import csv
import os

# Set print options
torch.set_printoptions(threshold=float('inf'))

model = "flame-moe-290m"
runid, epoch = 31066, 5473

# Parameters
sequence_length = 128
num_sequences_per_shard = 128
total_sequences_needed = 64  # ONLY load this many sequences

# Shard configuration
start_shard_idx = 2
num_shards = 1

# Base output directory
base_output_dir = Path("Encoder_Batch_2-64")
base_output_dir.mkdir(parents=True, exist_ok=True)

# Process layers
layers_to_process = list(range(2, 10))  # Layers 2-9

print(f"\n{'='*80}")
print(f"ENCODER TRACE EXTRACTION")
print(f"Processing ALL tokens from {total_sequences_needed} sequences")
print(f"Sequence length: {sequence_length} tokens")
print(f"Total tokens per layer: {total_sequences_needed * sequence_length}")
print(f"Shards to process: {num_shards} (starting from shard {start_shard_idx})")
print(f"{'='*80}\n")

# LOOP: Layers (one file per layer with ALL tokens from ALL sequences)
for layer in layers_to_process:
    print(f"\n{'='*80}")
    print(f"Processing LAYER {layer}")
    print(f"{'='*80}\n")
    
    # Create layer directory
    layer_dir = base_output_dir / f"Layer_{layer}"
    layer_dir.mkdir(parents=True, exist_ok=True)
    
    # Lists to collect ALL tokens from ALL sequences from ALL shards
    all_samples = []
    all_scores = []
    all_indices = []
    shard_list = []
    
    # Calculate sequences per shard (distribute evenly across shards)
    sequences_per_shard = total_sequences_needed // num_shards
    remaining_sequences = total_sequences_needed % num_shards
    
    # Process each shard
    sequences_collected = 0
    for idx, shard_idx in enumerate(range(start_shard_idx, start_shard_idx + num_shards)):
        shard = f"0-{shard_idx}.pt"
        shard_list.append(shard)
        
        # Calculate sequences to take from this shard
        seqs_from_this_shard = sequences_per_shard + (1 if idx < remaining_sequences else 0)
        tokens_from_this_shard = seqs_from_this_shard * sequence_length
        
        try:
            samples = torch.load(Path(f"samples/{model}/{runid}", shard), map_location="cpu")
            actives = torch.load(Path(f"actives/{model}/{runid}/{epoch}/{layer}", shard), map_location="cpu")
            scores, indices = actives
            
            # Only take the needed sequences from this shard
            samples_subset = samples[:tokens_from_this_shard]
            scores_subset = scores[:tokens_from_this_shard]
            indices_subset = indices[:tokens_from_this_shard]
            
            all_samples.append(samples_subset)
            all_scores.append(scores_subset)
            all_indices.append(indices_subset)
            
            sequences_collected += seqs_from_this_shard
            print(f"  Loaded shard {shard}: {len(samples_subset)} tokens from {seqs_from_this_shard} sequences")
            
        except FileNotFoundError:
            print(f"  Warning: Shard {shard} not found")
            break
    
    if len(all_samples) == 0:
        print(f"  No data found, skipping Layer {layer}")
        continue
    
    # Concatenate all shards
    samples_all = torch.cat(all_samples, dim=0)
    scores_all = torch.cat(all_scores, dim=0)
    indices_all = torch.cat(all_indices, dim=0)
    
    total_tokens = len(samples_all)
    total_sequences = total_tokens // sequence_length
    
    print(f"\n  Total tokens collected: {total_tokens}")
    print(f"  Total sequences: {total_sequences}")
    print(f"  Expected: {total_sequences_needed} sequences, {total_sequences_needed * sequence_length} tokens")
    
    # Verify we got the right amount
    if total_sequences != total_sequences_needed:
        print(f"  WARNING: Collected {total_sequences} sequences but expected {total_sequences_needed}!")
    
    # Create output filename
    first_shard_num = shard_list[0].replace('.pt', '')
    last_shard_num = shard_list[-1].replace('.pt', '')
    output_filename = (
        f"{model}_runid{runid}_epoch{epoch}_layer{layer}_shard{first_shard_num}_to_{last_shard_num}_encoder_{total_sequences}seqs_{total_tokens}tokens"
    )
    
    output_path_base = layer_dir / output_filename
    
    # Save .pt file
    output_data = {
        'model': model,
        'runid': runid,
        'epoch': epoch,
        'layer': layer,
        'num_sequences': total_sequences,
        'num_tokens': total_tokens,
        'sequence_length': sequence_length,
        'shards': shard_list,
        'samples': samples_all,
        'scores': scores_all,
        'indices': indices_all
    }
    torch.save(output_data, f'{output_path_base}.pt')
    
    # Generate CSV file
    csv_filename = f'{output_path_base}.csv'
    num_experts = 64
    
    print(f"  Generating CSV file...")
    with open(csv_filename, 'w', newline='') as csvfile:
        header = ['layer_id', 'token_id'] + [f'expert_{i}' for i in range(num_experts)]
        writer = csv.writer(csvfile)
        writer.writerow(header)
        
        for token_idx in range(len(samples_all)):
            row = [0, token_idx]
            expert_scores = [0.0] * num_experts
            
            for i in range(len(indices_all[token_idx])):
                expert_id = int(indices_all[token_idx][i].item())
                score = float(scores_all[token_idx][i].item())
                expert_scores[expert_id] = score
            
            formatted_scores = [f'{score:.6f}' for score in expert_scores]
            row.extend(formatted_scores)
            writer.writerow(row)
    
    # Generate statistics
    print(f"  Generating statistics...")
    df = pd.read_csv(csv_filename)
    expert_columns = [f'expert_{i}' for i in range(64)]
    
    # TOP 1
    expert_counts_top1 = {i: 0 for i in range(64)}
    for idx, row in df.iterrows():
        expert_scores = row[expert_columns].values.astype(float)
        top_1_index = np.argsort(expert_scores)[-1]
        expert_counts_top1[top_1_index] += 1
    
    sorted_experts_top1 = sorted(expert_counts_top1.items(), key=lambda x: x[1], reverse=True)
    expert_activ_top1 = sum(1 for count in expert_counts_top1.values() if count > 0)
    
    # TOP 2
    expert_counts_top2 = {i: 0 for i in range(64)}
    for idx, row in df.iterrows():
        expert_scores = row[expert_columns].values.astype(float)
        top_2_indices = np.argsort(expert_scores)[-2:][::-1]
        for expert_id in top_2_indices:
            expert_counts_top2[expert_id] += 1
    
    sorted_experts_top2 = sorted(expert_counts_top2.items(), key=lambda x: x[1], reverse=True)
    expert_activ_top2 = sum(1 for count in expert_counts_top2.values() if count > 0)
    
    # TOP 6
    expert_counts_top6 = {i: 0 for i in range(64)}
    for idx, row in df.iterrows():
        expert_scores = row[expert_columns].values.astype(float)
        top_6_indices = np.argsort(expert_scores)[-6:][::-1]
        for expert_id in top_6_indices:
            expert_counts_top6[expert_id] += 1
    
    sorted_experts_top6 = sorted(expert_counts_top6.items(), key=lambda x: x[1], reverse=True)
    expert_activ_top6 = sum(1 for count in expert_counts_top6.values() if count > 0)
    
    # Gating params
    BW_PCIe, BW_MD, alpha = 32, 512, 1.0
    
    expert_gpu_top1 = round((BW_PCIe / (BW_MD + BW_PCIe)) * expert_activ_top1)
    H_top1 = int(alpha * expert_gpu_top1)
    
    expert_gpu_top2 = round((BW_PCIe / (BW_MD + BW_PCIe)) * expert_activ_top2)
    H_top2 = int(alpha * expert_gpu_top2)
    
    expert_gpu_top6 = round((BW_PCIe / (BW_MD + BW_PCIe)) * expert_activ_top6)
    H_top6 = int(alpha * expert_gpu_top6)
    
    # Save stats files (TOP1, TOP2, TOP6)
    for top_k, sorted_experts, expert_activ, H in [
        (1, sorted_experts_top1, expert_activ_top1, H_top1),
        (2, sorted_experts_top2, expert_activ_top2, H_top2),
        (6, sorted_experts_top6, expert_activ_top6, H_top6)
    ]:
        stats_txt = f"{output_path_base}_top{top_k}_stats.txt"
        stats_csv = f"{output_path_base}_top{top_k}_stats.csv"
        
        with open(stats_txt, 'w') as f:
            f.write("=" * 60 + "\n")
            f.write(f"TOP {top_k} EXPERT TOKEN DISTRIBUTION (Layer {layer} - ALL SEQUENCES)\n")
            f.write("=" * 60 + "\n")
            f.write(f"Total tokens: {len(df)}\n")
            f.write(f"Total sequences: {total_sequences}\n")
            f.write(f"Sequence length: {sequence_length}\n")
            f.write(f"Expected total (tokens × {top_k}): {len(df) * top_k}\n")
            f.write("=" * 60 + "\n")
            f.write(f"{'Expert ID':<12} {'Token Count':<15} {'Percentage':<10}\n")
            f.write("-" * 60 + "\n")
            
            for expert_id, count in sorted_experts:
                percentage = (count / len(df)) * 100 if len(df) > 0 else 0.0
                f.write(f"Expert {expert_id:<5} {count:<15} {percentage:>6.2f}%\n")
            
            f.write("-" * 60 + "\n")
            f.write(f"\nGATING MECHANISM OUTPUT\n")
            f.write(f"Expert_Activ: {expert_activ}\n")
            f.write(f"H (Gating Output): {H}\n")
        
        with open(stats_csv, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Expert_ID', 'Token_Count', 'Percentage', 'Rank'])
            for rank, (expert_id, count) in enumerate(sorted_experts, 1):
                percentage = (count / len(df)) * 100 if len(df) > 0 else 0.0
                writer.writerow([expert_id, count, f"{percentage:.2f}", rank])
    
    print(f"  ✓ Saved Layer {layer}")
    print(f"    Files: .pt, .csv, _top1/2/6_stats (txt & csv)")
    print(f"    Total tokens in file: {total_tokens}")

print(f"\n{'='*80}")
print("All layers processed successfully!")
print(f"{'='*80}")


ENCODER TRACE EXTRACTION
Processing ALL tokens from 64 sequences
Sequence length: 128 tokens
Total tokens per layer: 8192
Shards to process: 1 (starting from shard 2)


Processing LAYER 2

  Loaded shard 0-2.pt: 8192 tokens from 64 sequences

  Total tokens collected: 8192
  Total sequences: 64
  Expected: 64 sequences, 8192 tokens
  Generating CSV file...
  Generating statistics...
  ✓ Saved Layer 2
    Files: .pt, .csv, _top1/2/6_stats (txt & csv)
    Total tokens in file: 8192

Processing LAYER 3

  Loaded shard 0-2.pt: 8192 tokens from 64 sequences

  Total tokens collected: 8192
  Total sequences: 64
  Expected: 64 sequences, 8192 tokens
  Generating CSV file...
  Generating statistics...
  ✓ Saved Layer 3
    Files: .pt, .csv, _top1/2/6_stats (txt & csv)
    Total tokens in file: 8192

Processing LAYER 4

  Loaded shard 0-2.pt: 8192 tokens from 64 sequences

  Total tokens collected: 8192
  Total sequences: 64
  Expected: 64 sequences, 8192 tokens
  Generating CSV file...
  Gene

In [67]:
import torch
from pathlib import Path
import shutil
import re

# Configuration
base_output_dir = Path("Encoder_Batch_2-64")

# Source and target layers
source_layers = list(range(2, 10))  # Layers 2-9
target_layers = list(range(10, 27))  # Layers 10-26

print(f"\n{'='*80}")
print(f"REPLICATING LAYERS 10-26 FROM LAYERS 2-9 (ENCODER)")
print(f"{'='*80}\n")

# For each target layer
for target_layer in target_layers:
    # Cycle through source layers (2-9)
    source_layer = source_layers[(target_layer - 10) % len(source_layers)]
    
    print(f"Creating Layer {target_layer} from Layer {source_layer}...")
    
    source_layer_dir = base_output_dir / f"Layer_{source_layer}"
    target_layer_dir = base_output_dir / f"Layer_{target_layer}"
    
    if not source_layer_dir.exists():
        print(f"  Warning: Source Layer_{source_layer} not found, skipping...")
        continue
    
    # Create target layer directory
    target_layer_dir.mkdir(parents=True, exist_ok=True)
    
    # Get all files from source layer
    source_files = list(source_layer_dir.glob("*"))
    
    if len(source_files) == 0:
        print(f"  Warning: No files in Layer_{source_layer}, skipping...")
        continue
    
    files_copied = 0
    
    for source_file in source_files:
        # Create new filename with updated layer number
        new_filename = source_file.name.replace(f"_layer{source_layer}_", f"_layer{target_layer}_")
        target_file = target_layer_dir / new_filename
        
        # Handle different file types
        if source_file.suffix == '.pt':
            # Load, update metadata, and save
            data = torch.load(source_file, map_location='cpu')
            data['layer'] = target_layer
            torch.save(data, target_file)
            files_copied += 1
            
        elif source_file.suffix == '.txt':
            # Read, replace layer references, and write
            with open(source_file, 'r') as f:
                content = f.read()
            
            # Replace layer number in multiple places
            content = content.replace(f'Layer: {source_layer}\n', f'Layer: {target_layer}\n')
            content = content.replace(f'(Layer {source_layer} - ALL SEQUENCES)', f'(Layer {target_layer} - ALL SEQUENCES)')
            content = content.replace(f'Layer {source_layer}', f'Layer {target_layer}')
            
            with open(target_file, 'w') as f:
                f.write(content)
            files_copied += 1
            
        elif source_file.suffix == '.csv':
            # CSV files can be copied directly (layer_id is already 0)
            shutil.copy2(source_file, target_file)
            files_copied += 1
            
        else:
            # For other files, just copy
            shutil.copy2(source_file, target_file)
            files_copied += 1
    
    print(f"  ✓ Layer {target_layer} created ({files_copied} files)")

print(f"\n{'='*80}")
print(f"All layers 10-26 replicated successfully!")
print(f"{'='*80}")
print(f"\nSummary:")
print(f"  - Created 17 new layers (10-26)")
print(f"  - Total layers: 25 (Layers 2-26)")
print(f"  - Each layer contains ALL tokens from ALL sequences")
print(f"  - Files per layer: .pt, .csv, _top1/2/6_stats (txt & csv)")


REPLICATING LAYERS 10-26 FROM LAYERS 2-9 (ENCODER)

Creating Layer 10 from Layer 2...
  ✓ Layer 10 created (8 files)
Creating Layer 11 from Layer 3...
  ✓ Layer 11 created (8 files)
Creating Layer 12 from Layer 4...
  ✓ Layer 12 created (8 files)
Creating Layer 13 from Layer 5...
  ✓ Layer 13 created (8 files)
Creating Layer 14 from Layer 6...
  ✓ Layer 14 created (8 files)
Creating Layer 15 from Layer 7...
  ✓ Layer 15 created (8 files)
Creating Layer 16 from Layer 8...
  ✓ Layer 16 created (8 files)
Creating Layer 17 from Layer 9...
  ✓ Layer 17 created (8 files)
Creating Layer 18 from Layer 2...
  ✓ Layer 18 created (8 files)
Creating Layer 19 from Layer 3...
  ✓ Layer 19 created (8 files)
Creating Layer 20 from Layer 4...
  ✓ Layer 20 created (8 files)
Creating Layer 21 from Layer 5...
  ✓ Layer 21 created (8 files)
Creating Layer 22 from Layer 6...
  ✓ Layer 22 created (8 files)
Creating Layer 23 from Layer 7...
  ✓ Layer 23 created (8 files)
Creating Layer 24 from Layer 8...
  ✓